In [ ]:
import pandas as pd
from tbdynamics.constants import (
    age_strata,
    organ_strata,
    compartments,

)
from tbdynamics.plotting import plot_model_vs_actual
import nevergrad as ng

# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.calib_utils import get_bcm, load_targets
from multiprocessing import cpu_count

In [ ]:
pd.options.plotting.backend = "plotly"

## Define Model

### Params and calibration targets

In [ ]:
# params = {
#     'treatment_duration': 0.5, # 6 months
#     'screening_start_asymp': 0.,
# }

bcm = get_bcm()

### Running Optimization

In [ ]:
# Set the number of workers for parallel optimization
num_workers = cpu_count()
opt_class = ng.optimizers.TwoPointsDE
orunner = optimize_model(bcm, opt_class=opt_class, num_workers=num_workers)
for i in range(8):
    rec = orunner.minimize(10)
mle_params = rec.value[1]
mle_params
   

In [ ]:
mle_params = {'start_population_size': 2061737.729729285,
 'contact_rate': 0.004974794112768116,
 'rr_infection_latent': 0.4659015958273958,
 'rr_infection_recovered': 0.3510972543223931,
 'progression_multiplier': 1.185884064749275,
 'seed_time': 1890.2820370624606,
 'seed_num': 44.25246661729487,
 'seed_duration': 4.696723206900956,
 'smear_positive_death_rate': 0.3982500113327807,
 'smear_negative_death_rate': 0.019807391194471922,
 'smear_positive_self_recovery': 0.20442159021310669,
 'smear_negative_self_recovery': 0.18053417169001693,
 'screening_scaleup_shape': 0.09024357865291716,
 'screening_inflection_time': 1993.0721755530203,
 'screening_end_asymp': 0.5964493661599798}

In [ ]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets()

### Outputs

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['pop'], "total_population", "Population", "Modelled vs Data"
)

In [ ]:
derived_df_0[[f"total_populationXage_{i}" for i in age_strata]].plot(
    title="Modelled populatation by age group", kind="area"
)

In [ ]:

plot_model_vs_actual(
    derived_df_0, targets['incidence'], "incidence", "Incidence", "Modelled vs Data"
)

In [ ]:
derived_df_0[[f"prop_{compartment}" for compartment in compartments]].plot(kind="area")

In [ ]:
derived_df_0[[f"total_populationXorgan_{i}" for i in organ_strata]].plot(
    title="Modelled populatation by organ status", kind="area"
)

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['notifs'], "notification", "Notification", "Modelled vs Data"
)

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['percentage_latent'], "percentage_latent", "Percentage latent", "Modelled vs Data"
)

In [ ]:
derived_df_0['cdr'].plot()

In [ ]:
plot_model_vs_actual(derived_df_0, targets['prevalence_pulmonary'], 'prevalence_pulmonary', 'Infectious prevalence', 'Modelled vs Estimation from 2017 prevalence survey')
